In [27]:
import pandas as pd
import numpy as np

In [28]:
train = pd.read_csv('/Users/caleboneel/Downloads/us-train.csv')
val = pd.read_csv('/Users/caleboneel/Downloads/us-val.csv')
us = pd.read_csv('/Users/caleboneel/Downloads/us-test.csv')

In [29]:
frames = [train, val]
X = pd.concat(frames)


In [30]:
y_train = train.log_price
y_val = val.log_price
y = X.log_price

X_train = train.drop(columns=['log_price'])
X_val = val.drop(columns=['log_price'])
X = X.drop(columns=['log_price'])

In [12]:

X_train.fillna(0, inplace=True)
X_train.replace([np.inf, -np.inf], 0, inplace=True)
X_val.fillna(0, inplace=True)
X_val.replace([np.inf, -np.inf], 0, inplace=True)

#### Lasso Regression Random Search

In [13]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from time import time
import scipy.stats as stats
from sklearn.utils.fixes import loguniform
from tqdm import tqdm
from sklearn import linear_model
from sklearn.model_selection import RepeatedKFold
from numpy import absolute


# Create MLP model with desired parameters, omitting the ones we will use random search to find
clf = linear_model.Lasso()
kfold = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

# A function to print results of random search
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})"
                  .format(results['mean_test_score'][candidate],
                          results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")
            
# specify parameters and distributions to sample from
param_dist = {
              'alpha': loguniform(1e-8, 1e2),
           }

# run randomized search
n_iter_search = 30
random_search = RandomizedSearchCV(clf, param_distributions=param_dist, cv=5, n_jobs=-1,
                                   n_iter=n_iter_search, verbose=1)

# Time and output results of random search model
start = time()
random_search.fit(X_train, y_train)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   55.4s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  3.3min finished


RandomizedSearchCV took 205.07 seconds for 30 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.590 (std: 0.009)
Parameters: {'alpha': 2.3976811762816578e-06}

Model with rank: 2
Mean validation score: 0.590 (std: 0.009)
Parameters: {'alpha': 5.557750095611968e-05}

Model with rank: 3
Mean validation score: 0.590 (std: 0.009)
Parameters: {'alpha': 5.613367916580479e-05}



/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1943.2039547093468, tolerance: 2.2893091545390534
  model = cd_fast.enet_coordinate_descent(


In [16]:
lasso_opt = linear_model.Lasso(alpha = 5.557750095611968e-05).fit(X_train, y_train)

In [33]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error
import time

In [19]:
y_pred_train = lasso_opt.predict(X_train)
y_pred = lasso_opt.predict(X_val)

acc_train = lasso_opt.score(X_train, y_train)
acc = lasso_opt.score(X_val, y_val)

rmse_train = mean_squared_error(np.exp(y_train), np.exp(y_pred_train), squared=False)
rmse = mean_squared_error(np.exp(y_val), np.exp(y_pred), squared=False)

mae_train = mean_absolute_error(np.exp(y_train), np.exp(y_pred_train))
mae = mean_absolute_error(np.exp(y_val), np.exp(y_pred))

medae_train = median_absolute_error(np.exp(y_train), np.exp(y_pred_train))
medae = median_absolute_error(np.exp(y_val), np.exp(y_pred))



print('R^2 (Train):', round(acc_train,2)) 
print('R^2 (Val):', round(acc,2))
print('')
print('RMSE (Train):', round(rmse_train,2))
print('RMSE (Val):', round(rmse,2))
print('')
print('Mean Abs Error (Train):', round(mae_train,2))
print('Mean Abs Error (Val):', round(mae,2))
print('')
print('Median Absolute Error (Train):', round(medae_train,2))
print('Median Absolute Error (Val):', round(medae,2))

R^2 (Train): 0.59
R^2 (Val): 0.59

RMSE (Train): 130.44
RMSE (Val): 130.04

Mean Abs Error (Train): 59.0
Mean Abs Error (Val): 59.08

Median Absolute Error (Train): 28.3
Median Absolute Error (Val): 28.4


#### US and EU Test

In [20]:
london = pd.read_csv('/Users/caleboneel/Downloads/london-test.csv')
madrid = pd.read_csv('/Users/caleboneel/Downloads/madrid-test.csv')
paris = pd.read_csv('/Users/caleboneel/Downloads/paris-test.csv')

In [24]:
y_test = us.log_price
X_test = us.drop(columns=['log_price'])

In [31]:
lasso_final = linear_model.Lasso(alpha = 5.557750095611968e-05).fit(X, y)

In [51]:
y_pred_train = lasso_final.predict(X)
test_start = time.time()
y_pred = lasso_final.predict(X_test)
test_end = time.time()

acc_train = lasso_final.score(X, y)
acc = lasso_final.score(X_test, y_test)

rmse_train = mean_squared_error(np.exp(y), np.exp(y_pred_train), squared=False)
rmse = mean_squared_error(np.exp(y_test), np.exp(y_pred), squared=False)

mae_train = mean_absolute_error(np.exp(y), np.exp(y_pred_train))
mae = mean_absolute_error(np.exp(y_test), np.exp(y_pred))

medae_train = median_absolute_error(np.exp(y), np.exp(y_pred_train))
medae = median_absolute_error(np.exp(y_test), np.exp(y_pred))



NameError: name 'time_end' is not defined

In [35]:
print('R^2 (Train):', round(acc_train,2)) 
print('R^2 (Test):', round(acc,2))
print('')
print('RMSE (Train):', round(rmse_train,2))
print('RMSE (Test):', round(rmse,2))
print('')
print('Mean Abs Error (Train):', round(mae_train,2))
print('Mean Abs Error (Test):', round(mae,2))
print('')
print('Median Absolute Error (Train):', round(medae_train,2))
print('Median Absolute Error (Test):', round(medae,2))

R^2 (Train): 0.59
R^2 (Test): 0.58

RMSE (Train): 130.35
RMSE (Test): 132.79

Mean Abs Error (Train): 59.02
Mean Abs Error (Test): 59.5

Median Absolute Error (Train): 28.34
Median Absolute Error (Test): 28.61


### Eur

In [39]:
madrid_y = madrid.log_price
madrid_X = madrid.drop(columns=['log_price'])

london_y = london.log_price
london_X = london.drop(columns=['log_price'])

paris_y = paris.log_price
paris_X = paris.drop(columns=['log_price'])

In [40]:
from sklearn.metrics import r2_score

madrid_col = 100/75.8
madrid_y = madrid_col*madrid_y

In [42]:
mad_start = time.time()
y_pred = lasso_final.predict(madrid_X)
mad_end = time.time()

acc = lasso_final.score(madrid_X, madrid_y)

rmse = mean_squared_error(np.exp(madrid_y), np.exp(y_pred), squared=False)

mae = mean_absolute_error(np.exp(madrid_y), np.exp(y_pred))

medae = median_absolute_error(np.exp(madrid_y), np.exp(y_pred))



print('Test Time (madrid):', round(mad_end-mad_start,2))
print('')
print('R^2 (Madrid):', round(acc,2))
print('')
print('RMSE (Madrid):', round(rmse,2))
print('')
print('Mean Abs Error (Madrid):', round(mae,2))
print('')
print('Median Absolute Error (Madrid):', round(medae,2))

Test Time (madrid): 0.01

R^2 (Madrid): 0.01

RMSE (Madrid): 7780.96

Mean Abs Error (Madrid): 889.05

Median Absolute Error (Madrid): 73.24


In [43]:
london_col = 100/95
london_y = london_col*london_y

In [45]:

lon_start = time.time()
y_pred = lasso_final.predict(london_X)
lon_end = time.time()

acc = lasso_final.score(london_X, london_y)

rmse = mean_squared_error(np.exp(london_y), np.exp(y_pred), squared=False)

mae = mean_absolute_error(np.exp(london_y), np.exp(y_pred))

medae = median_absolute_error(np.exp(london_y), np.exp(y_pred))


print('Test Time (London):', round(lon_end-lon_start,2))
print('')
print('R^2 (London):', round(acc,2))
print('')
print('RMSE (London):', round(rmse,2))
print('')
print('Mean Abs Error (London):', round(mae,2))
print('')
print('Median Absolute Error (London):', round(medae,2))

Test Time (London): 0.04

R^2 (London): 0.19

RMSE (London): 176437.65

Mean Abs Error (London): 739.63

Median Absolute Error (London): 55.66


### Paris

In [46]:
france_col = 100/89.3
paris_y = france_col*paris_y

In [48]:
paris_start = time.time()
y_pred = lasso_final.predict(paris_X)
paris_end = time.time()

acc = lasso_final.score(paris_X, paris_y)

rmse = mean_squared_error(np.exp(paris_y), np.exp(y_pred), squared=False)

mae = mean_absolute_error(np.exp(paris_y), np.exp(y_pred))

medae = median_absolute_error(np.exp(paris_y), np.exp(y_pred))



print('Test Time (Paris):', round(paris_end-paris_start,2))
print('')
print('R^2 (Paris):', round(acc,2))
print('')
print('RMSE (Paris):', round(rmse,2))
print('')
print('Mean Abs Error (Paris):', round(mae,2))
print('')
print('Median Absolute Error (Paris):', round(medae,2))

Test Time (Paris): 0.2

R^2 (Paris): 0.17

RMSE (Paris): 918898.77

Mean Abs Error (Paris): 5164.26

Median Absolute Error (Paris): 59.8


#### All EU

In [49]:
international_X = pd.concat([madrid_X, london_X, paris_X])
international_y = pd.concat([madrid_y, london_y, paris_y])


In [50]:

int_start = time.time()
y_pred = lasso_final.predict(international_X)
int_end = time.time()

acc = lasso_final.score(international_X, international_y)

rmse = mean_squared_error(np.exp(international_y), np.exp(y_pred), squared=False)

mae = mean_absolute_error(np.exp(international_y), np.exp(y_pred))

medae = median_absolute_error(np.exp(international_y), np.exp(y_pred))


print('Test Time (Int):', round(int_end-int_start,2))
print('')
print('R^2 (Int):', round(acc,2))
print('')
print('RMSE (Int):', round(rmse,2))
print('')
print('Mean Abs Error (Int):', round(mae,2))
print('')
print('Median Absolute Error (Int):', round(medae,2))

Test Time (Int): 0.57

R^2 (Int): 0.24

RMSE (Int): 594236.89

Mean Abs Error (Int): 2531.26

Median Absolute Error (Int): 58.12


In [54]:
international_X

,accommodates,bathrooms,host_has_profile_pic,host_identity_verified,host_response_rate,instant_bookable,number_of_reviews,review_scores_rating,thumbnail_url,bedrooms,...,x0_Tipi,x0_Townhouse,x0_Train,x0_Treehouse,x0_Vacation home,x0_Villa,x0_Yurt,x1_Entire home/apt,x1_Private room,x1_Shared room
0,2,1.0,1,1,100.000000,0,78,98.000000,1,1.0,...,0,0.0,0,0,0,0.0,0.0,0.0,1.0,0.0
1,1,1.0,1,1,83.129967,0,33,92.000000,1,1.0,...,0,0.0,0,0,0,0.0,0.0,0.0,1.0,0.0
2,6,2.0,1,1,89.000000,0,0,92.569712,1,3.0,...,0,0.0,0,0,0,0.0,0.0,1.0,0.0,0.0
3,3,1.0,1,1,100.000000,1,10,98.000000,1,1.0,...,0,0.0,0,0,0,0.0,0.0,1.0,0.0,0.0
4,4,1.5,1,0,83.129967,0,68,98.000000,1,2.0,...,0,0.0,0,0,0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64623,2,1.0,1,1,82.000000,1,0,93.063931,1,1.0,...,0,0.0,0,0,0,0.0,0.0,1.0,0.0,0.0
64624,2,1.0,1,1,100.000000,1,0,93.063931,1,1.0,...,0,0.0,0,0,0,0.0,0.0,1.0,0.0,0.0
64625,2,1.0,1,1,100.000000,0,0,93.063931,1,1.0,...,0,0.0,0,0,0,0.0,0.0,1.0,0.0,0.0
64626,1,1.0,1,1,100.000000,0,0,93.063931,1,1.0,...,0,0.0,0,0,0,0.0,0.0,1.0,0.0,0.0
